In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from data_input import LoadData
from scipy import stats

In [2]:
data_dir = '/mnt/c/senior_project/data'
df_arr = []
control_arr = []
session_folders = []
skilled_session = ['session_2', 'session_3', 'session_4', 'session_8', 'session_9', 'session_11']
session_ids = [10, 11, 12, 13, 2, 3, 4, 5, 6, 7, 8, 9]

In [3]:
# NOTE: file structure:
#   -/ data
#       -/control
#           -/ session_#
#               - files
#       -/participants
#           -/ session_#
#               - files
session_folders = []
for _, folders, _ in os.walk(data_dir):
    for player_type in folders:
        player_dir = os.path.join(data_dir, player_type)
        for path, folders, files in os.walk(player_dir):
            for folder in folders:
                folder_dir = os.path.join(player_dir, folder) 
                session_folders.append(folder)
                for path, folders, files in os.walk(folder_dir):
                    hr_file = os.path.join(folder_dir, files[2])
                    timestamp_file = os.path.join(folder_dir, files[3])
                    tracking_file = os.path.join(folder_dir, files[4])
                    if folder in skilled_session or player_type == 'control':
                        skilled_flag = True
                    else:
                        skilled_flag = False
                    if player_type == 'control':
                        if folder == 'session_13':
                            control_arr.append(LoadData(hr_file, tracking_file, skilled_flag, hours=-21.53611).dataframe)
                        else:
                            control_arr.append(LoadData(hr_file, tracking_file, skilled_flag, hours=7).dataframe)
                    else:
                        df_arr.append(LoadData(hr_file, tracking_file, skilled_flag, hours=7).dataframe)
print(session_folders)

['session_10', 'session_11', 'session_12', 'session_13', 'session_2', 'session_3', 'session_4', 'session_5', 'session_6', 'session_7', 'session_8', 'session_9', 'session_10', 'session_11', 'session_12', 'session_13', 'session_2', 'session_3', 'session_4', 'session_5', 'session_6', 'session_7', 'session_8', 'session_9']


In [4]:
i = 0
for df in control_arr:
    print(f"Session {session_ids[i]} Dataframe")
    i += 1
    print(df)

Session 10 Dataframe
    Interval         HR  APM  Importance   Base HR  Label
0          0  83.193077   60           0  81.98913      1
1          1  80.647500  282           0  81.98913      1
2          2  64.855714   12           0  81.98913      1
3          3  70.614286   12           1  81.98913      1
4          4  68.346000    0           0  81.98913      1
5          5  79.384000    6           0  81.98913      1
6          6  75.194545  204           0  81.98913      1
7          7  74.917500   12           1  81.98913      1
8          8  67.988889   18           0  81.98913      1
9          9  67.535714   84           2  81.98913      1
10        10  66.560000    6           1  81.98913      1
11        11  70.362500   36           0  81.98913      1
12        12  70.590000   36           0  81.98913      1
13        13  77.814286   30           1  81.98913      1
14        14  80.314000  156           2  81.98913      1
15        15  74.692500  192           2  81.98913 

In [5]:
i = 0
for df in df_arr:
    print(f"{session_folders[i]} Dataframe")
    i += 1
    print(df)

session_10 Dataframe
    Interval          HR  APM  Importance    Base HR  Label
0          0  103.500000    0           0  99.597455      0
1          1  102.167000    0           0  99.597455      0
2          2   77.882222    0           0  99.597455      0
3          3   93.507273   18           1  99.597455      0
4          4   84.485455   36           0  99.597455      0
5          5  103.463846  390           0  99.597455      0
6          6  105.291000   18           0  99.597455      0
7          7   90.891818   12           1  99.597455      0
8          8   69.440000   30           0  99.597455      0
9          9   87.947273   60           2  99.597455      0
10        10  102.586364   48           1  99.597455      0
11        11  126.890000   24           0  99.597455      0
12        12  106.355556   48           0  99.597455      0
13        13   68.810000   42           1  99.597455      0
session_11 Dataframe
    Interval          HR  APM  Importance    Base HR  Labe

In [6]:
# Create Separate Dataframes for each player type
control_df = pd.concat(control_arr)
player_df = pd.concat(df_arr)
unskilled_df = player_df.loc[player_df['Label'] == 0]
skilled_df = player_df[player_df['Label'] == 1]

Check Statistical Significance

In [ ]:
# dataset as a whole
anova_result = stats.f_oneway(control_df, unskilled_df, skilled_df)
p_value_dict = {'HR': anova_result.pvalue[1], 'APM': anova_result.pvalue[2]}
print(p_value_dict)

{'HR': np.float64(0.004804799489466654), 'APM': np.float64(0.0319382805121749)}


/mnt/c/senior_project/env/lib/python3.10/site-packages/scipy/stats/_axis_nan_policy.py:605: ConstantInputWarning: Each of the input arrays is constant; the F statistic is not defined or infinite
  res = hypotest_fun_out(*samples, axis=axis, **kwds)


In [ ]:
# APM during high importance event
event3_APM_s = skilled_df.loc[skilled_df['Importance'] == 2]
event3_APM_s = event3_APM_s['APM']
event3_APM_u = unskilled_df.loc[unskilled_df['Importance'] == 2]
event3_APM_u = event3_APM_u['APM']

print(stats.f_oneway(event3_APM_s, event3_APM_u).pvalue)

0.01397669261329763


In [ ]:
# HR during high importance event
event3_HR_s = skilled_df.loc[skilled_df['Importance'] == 2]
event3_HR_s = event3_HR_s['HR']
event3_HR_u = unskilled_df.loc[unskilled_df['Importance'] == 2]
event3_HR_u = event3_HR_u['HR']

print(stats.f_oneway(event3_HR_s, event3_HR_u).pvalue)

0.04223589457899677


In [ ]:
# APM during Medium importance event
event2_APM_s = skilled_df.loc[skilled_df['Importance'] == 1]
event2_APM_s = event2_APM_s['APM']
event2_APM_u = unskilled_df.loc[unskilled_df['Importance'] == 1]
event2_APM_u = event2_APM_u['APM']

print(stats.f_oneway(event2_APM_s, event2_APM_u).pvalue)

0.05354698221163798


In [18]:
# HR during medium importance event
event2_HR_s = skilled_df.loc[skilled_df['Importance'] == 1]
event2_HR_s = event2_HR_s['HR']
event2_HR_u = unskilled_df.loc[unskilled_df['Importance'] == 1]
event2_HR_u = event2_HR_u['HR']

print(stats.f_oneway(event2_HR_s, event2_HR_u).pvalue)

0.29710657186581113


In [20]:
# APM during low importance event
event1_APM_s = skilled_df.loc[skilled_df['Importance'] == 0]
event1_APM_s = event1_APM_s['APM']
event1_APM_u = unskilled_df.loc[unskilled_df['Importance'] == 0]
event1_APM_u = event1_APM_u['APM']

print(stats.f_oneway(event1_APM_s, event1_APM_u).pvalue)

0.36575353892068974


In [21]:
# HR during low importance event
event1_HR_s = skilled_df.loc[skilled_df['Importance'] == 0]
event1_HR_s = event1_HR_s['HR']
event1_HR_u = unskilled_df.loc[unskilled_df['Importance'] == 0]
event1_HR_u = event1_HR_u['HR']

print(stats.f_oneway(event1_HR_s, event1_HR_u).pvalue)

0.006465423847453927


In [23]:
# HR overall
skilled_HR = skilled_df['HR']
unskilled_HR = unskilled_df['HR']

print(stats.f_oneway(skilled_HR, unskilled_HR).pvalue)

0.0005969034488825274


In [24]:
# APM overall
skilled_APM = skilled_df['APM']
unskilled_APM = unskilled_df['APM']

print(stats.f_oneway(skilled_APM, unskilled_APM).pvalue)

0.01428948070038016
